# 🌐 Universal Translator - AI-Powered Multi-Language Translation System

## 1. Problem Definition & Objective

### a. Selected Project Track
**AI/NLP - Machine Translation System**

### b. Clear Problem Statement
Building a universal translation system that can accurately translate text between 20+ languages using multiple translation engines with intelligent fallback mechanisms.

### c. Real-world Relevance and Motivation
- Breaking language barriers in global communication
- Supporting multilingual applications and services
- Providing reliable translation with multiple engine options
- Offering caching for performance optimization
- Creating an interactive Colab-ready interface for accessibility

## 2. Data Understanding & Preparation

### a. Dataset Source
- **Pre-trained Models**: Utilizing transformer-based models from Hugging Face
- **Language Models**: NLLB-200, Helsinki OPUS, mBART-50
- **Language Detection**: Using `langdetect` library for automatic source language identification

### b. Data Loading and Exploration
The system loads pre-trained translation models on-demand and handles text input through an interactive interface.

In [4]:
# Install required dependencies
!pip -q install -U transformers accelerate sentencepiece sacremoses langdetect langcodes language_data ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 131.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 96.5 MB/s eta 0:00:00


### c. Import Libraries and Setup

In [5]:
import time
import json
import torch
import warnings
from datetime import datetime
from typing import Dict, List, Optional, Tuple

warnings.filterwarnings("ignore")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

import langcodes
import ipywidgets as widgets
from IPython.display import display, clear_output

## 3. Model / System Design

### a. AI Technique Used
**Natural Language Processing (NLP) - Neural Machine Translation**

### b. Architecture Explanation
The system uses a multi-engine approach with intelligent fallback:
1. **Helsinki OPUS Models**: Fast, specialized bilingual models
2. **NLLB-200**: Facebook's No Language Left Behind model for 200+ languages
3. **mBART-50**: Multilingual BART model for 50 languages

### c. Justification of Design Choices
- **Multi-engine approach**: Ensures availability for different language pairs
- **Caching system**: Improves performance for repeated translations
- **Auto language detection**: Simplifies user experience
- **GPU optimization**: Accelerates inference when available
- **Interactive widget**: Makes it accessible in Colab/Jupyter environments

### Language Configuration and Mapping

In [6]:
# Language mapping with NLLB codes, names, and emojis for UI
LANGUAGE_MAPPING = {
    "auto": {"code": "auto", "name": "Auto Detect", "emoji": "🔍"},
    "english": {"code": "eng_Latn", "name": "English", "emoji": "🇬🇧"},
    "spanish": {"code": "spa_Latn", "name": "Spanish", "emoji": "🇪🇸"},
    "french": {"code": "fra_Latn", "name": "French", "emoji": "🇫🇷"},
    "german": {"code": "deu_Latn", "name": "German", "emoji": "🇩🇪"},
    "chinese": {"code": "zho_Hans", "name": "Chinese (Simplified)", "emoji": "🇨🇳"},
    "arabic": {"code": "arb_Arab", "name": "Arabic", "emoji": "🇸🇦"},
    "hindi": {"code": "hin_Deva", "name": "Hindi", "emoji": "🇮🇳"},
    "russian": {"code": "rus_Cyrl", "name": "Russian", "emoji": "🇷🇺"},
    "japanese": {"code": "jpn_Jpan", "name": "Japanese", "emoji": "🇯🇵"},
    "portuguese": {"code": "por_Latn", "name": "Portuguese", "emoji": "🇵🇹"},
    "italian": {"code": "ita_Latn", "name": "Italian", "emoji": "🇮🇹"},
    "dutch": {"code": "nld_Latn", "name": "Dutch", "emoji": "🇳🇱"},
    "korean": {"code": "kor_Hang", "name": "Korean", "emoji": "🇰🇷"},
    "turkish": {"code": "tur_Latn", "name": "Turkish", "emoji": "🇹🇷"},
    "vietnamese": {"code": "vie_Latn", "name": "Vietnamese", "emoji": "🇻🇳"},
    "thai": {"code": "tha_Thai", "name": "Thai", "emoji": "🇹🇭"},
    "swahili": {"code": "swh_Latn", "name": "Swahili", "emoji": "🇰🇪"},
    "urdu": {"code": "urd_Arab", "name": "Urdu", "emoji": "🇵🇰"},
    "persian": {"code": "pes_Arab", "name": "Persian", "emoji": "🇮🇷"},
    "bengali": {"code": "ben_Beng", "name": "Bengali", "emoji": "🇧🇩"},
}

### Helsinki OPUS Model Registry
Specialized bilingual models for common language pairs

In [7]:
HELSINKI_MODELS = {
    "en-es": "Helsinki-NLP/opus-mt-en-es",
    "es-en": "Helsinki-NLP/opus-mt-es-en",
    "en-fr": "Helsinki-NLP/opus-mt-en-fr",
    "fr-en": "Helsinki-NLP/opus-mt-fr-en",
    "en-de": "Helsinki-NLP/opus-mt-en-de",
    "de-en": "Helsinki-NLP/opus-mt-de-en",
    "en-ru": "Helsinki-NLP/opus-mt-en-ru",
    "ru-en": "Helsinki-NLP/opus-mt-ru-en",
    "en-ar": "Helsinki-NLP/opus-mt-en-ar",
    "ar-en": "Helsinki-NLP/opus-mt-ar-en",
}

## 4. Core Implementation

### a. Translation Cache Class
Implements caching mechanism to store and retrieve translations for performance optimization

In [8]:
class TranslationCache:
    """Cache system to store translations and improve performance"""
    def __init__(self):
        self.cache = {}
        self.stats = {"hits": 0, "misses": 0, "size": 0}

    def _key(self, text: str, src: str, tgt: str, engine: str) -> str:
        """Generate unique cache key"""
        return f"{hash(text)}::{src}::{tgt}::{engine}"

    def get(self, text: str, src: str, tgt: str, engine: str):
        """Retrieve from cache if exists"""
        k = self._key(text, src, tgt, engine)
        if k in self.cache:
            self.stats["hits"] += 1
            return self.cache[k]
        self.stats["misses"] += 1
        return None

    def put(self, text: str, src: str, tgt: str, engine: str, result: Dict):
        """Store translation result in cache"""
        k = self._key(text, src, tgt, engine)
        self.cache[k] = result
        self.stats["size"] = len(self.cache)

    def clear(self):
        """Clear all cached translations"""
        self.cache.clear()
        self.stats = {"hits": 0, "misses": 0, "size": 0}

    def get_stats(self):
        """Get cache statistics"""
        total = self.stats["hits"] + self.stats["misses"]
        eff = (self.stats["hits"] / total * 100) if total else 0.0
        return {**self.stats, "efficiency": eff}

### b. Main Translator Class
Orchestrates multiple translation engines with intelligent fallback

In [9]:
class LanguageTranslator:
    """Main translation orchestrator with multi-engine support"""
    def __init__(self):
        self.device = 0 if torch.cuda.is_available() else -1
        self.cache = TranslationCache()

        # Lazy loaded pipelines
        self._helsinki_pipe = None
        self._helsinki_model_id = None
        self._nllb_pipe = None
        self._mbart_pipe = None

    def display_device_info(self):
        """Display GPU/CPU information"""
        if torch.cuda.is_available():
            print("🟢 GPU Enabled:", torch.cuda.get_device_name(0))
        else:
            print("🟡 Running on CPU")

    def detect_language(self, text: str) -> Tuple[str, str]:
        """Detect language of input text"""
        try:
            if len(text.strip()) < 3:
                return "en", "English"
            code = detect(text)
            name = langcodes.Language.get(code).display_name()
            return code, name
        except:
            return "en", "English"

    def _load_nllb(self):
        """Lazy load NLLB-200 model"""
        if self._nllb_pipe is None:
            model_id = "facebook/nllb-200-distilled-600M"
            self._nllb_pipe = pipeline(
                "translation",
                model=model_id,
                device=self.device,
                torch_dtype=torch.float16 if torch.cuda.is_available() else None,
            )
        return self._nllb_pipe

    def _load_mbart(self):
        """Lazy load mBART-50 model"""
        if self._mbart_pipe is None:
            model_id = "facebook/mbart-large-50-many-to-many-mmt"
            self._mbart_pipe = pipeline(
                "translation",
                model=model_id,
                device=self.device,
                torch_dtype=torch.float16 if torch.cuda.is_available() else None,
            )
        return self._mbart_pipe

    def _load_helsinki(self, model_id: str):
        """Lazy load Helsinki OPUS model"""
        if self._helsinki_pipe is None or self._helsinki_model_id != model_id:
            self._helsinki_pipe = pipeline(
                "translation",
                model=model_id,
                device=self.device,
            )
            self._helsinki_model_id = model_id
        return self._helsinki_pipe

    def translate(
        self,
        text: str,
        target_language: str = "spanish",
        source_language: str = "auto",
        engine: str = "auto",   # auto | nllb | helsinki | mbart
        max_length: int = 512,
    ) -> Dict:
        """Main translation method with multi-engine fallback"""

        if not text.strip():
            return {"error": "Empty input text"}

        # Check cache first
        cached = self.cache.get(text, source_language, target_language, engine)
        if cached:
            cached["cached"] = True
            return cached

        # Detect source language if auto
        if source_language == "auto":
            src_code_short, src_name = self.detect_language(text)
        else:
            src_code_short = source_language
            src_name = source_language

        # Map to NLLB language codes
        src_info = LANGUAGE_MAPPING.get(source_language.lower(), LANGUAGE_MAPPING["english"])
        tgt_info = LANGUAGE_MAPPING.get(target_language.lower(), LANGUAGE_MAPPING["spanish"])

        src_nllb = src_info["code"]
        tgt_nllb = tgt_info["code"]

        # Choose engine with intelligent fallback
        used_engine = None
        translated = None

        # 1) Helsinki OPUS (fast for supported pairs)
        if engine in ("auto", "helsinki"):
            src_prefix = src_nllb.split("_")[0]
            tgt_prefix = tgt_nllb.split("_")[0]
            key = f"{src_prefix}-{tgt_prefix}"

            if key in HELSINKI_MODELS:
                try:
                    pipe = self._load_helsinki(HELSINKI_MODELS[key])
                    out = pipe(text, max_length=max_length)
                    translated = out[0]["translation_text"]
                    used_engine = "helsinki"
                except Exception as e:
                    translated = None

        # 2) NLLB-200 fallback (best universal coverage)
        if translated is None and engine in ("auto", "nllb"):
            try:
                pipe = self._load_nllb()
                out = pipe(
                    text,
                    src_lang=src_nllb if src_nllb != "auto" else "eng_Latn",
                    tgt_lang=tgt_nllb,
                    max_length=max_length,
                )
                translated = out[0]["translation_text"]
                used_engine = "nllb"
            except Exception as e:
                translated = None

        # 3) mBART-50 fallback (alternative multilingual)
        if translated is None and engine in ("auto", "mbart"):
            try:
                pipe = self._load_mbart()
                out = pipe(text, max_length=max_length)
                translated = out[0]["translation_text"]
                used_engine = "mbart"
            except Exception:
                translated = None

        if translated is None:
            translated = "Translation failed. Try a different language pair or engine."
            used_engine = "failed"

        # Prepare result dictionary
        result = {
            "original_text": text,
            "translated_text": translated,
            "source_language": src_name.title(),
            "target_language": target_language.title(),
            "engine_used": used_engine,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "characters": len(text),
            "cached": False,
            "src_emoji": src_info.get("emoji", ""),
            "tgt_emoji": tgt_info.get("emoji", ""),
        }

        # Cache the result
        self.cache.put(text, source_language, target_language, engine, result)
        return result

    def batch_translate(self, texts: List[str], target_language: str, source_language="auto", engine="auto"):
        """Translate multiple texts sequentially"""
        out = []
        for t in texts:
            out.append(self.translate(t, target_language, source_language, engine))
        return out

### c. System Initialization and Setup

In [10]:
# Initialize the translator
translator = LanguageTranslator()
translator.display_device_info()
print("✅ Translator Ready")

🟢 GPU Enabled: Tesla T4
✅ Translator Ready


### d. Interactive User Interface
Creating a widget-based interface for easy interaction in Colab/Jupyter

In [11]:
def create_widget():
    """Create interactive translation widget"""
    input_text = widgets.Textarea(
        value="Hello! How are you today?",
        description="📝 Input:",
        layout=widgets.Layout(width="95%", height="140px"),
        style={"description_width": "initial"},
    )

    # Prepare language options for dropdown
    language_options = [
        (f"{LANGUAGE_MAPPING[k]['emoji']} {LANGUAGE_MAPPING[k]['name']}", k)
        for k in LANGUAGE_MAPPING
    ]

    source_lang = widgets.Dropdown(
        options=language_options,
        value="auto",
        description="🌍 From:",
        style={"description_width": "initial"},
    )

    target_lang = widgets.Dropdown(
        options=[x for x in language_options if x[1] != "auto"],
        value="spanish",
        description="🎯 To:",
        style={"description_width": "initial"},
    )

    engine = widgets.Dropdown(
        options=[("Auto", "auto"), ("NLLB-200", "nllb"), ("Helsinki OPUS", "helsinki"), ("mBART-50", "mbart")],
        value="auto",
        description="🤖 Engine:",
        style={"description_width": "initial"},
    )

    # Action buttons
    btn = widgets.Button(description="✨ Translate", button_style="success")
    clear_btn = widgets.Button(description="🗑️ Clear", button_style="warning")
    cache_btn = widgets.Button(description="💾 Clear Cache", button_style="info")

    output = widgets.Output(layout={"border": "1px solid #ccc", "padding": "10px"})
    cache_info = widgets.HTML()

    def refresh_cache():
        """Update cache statistics display"""
        s = translator.cache.get_stats()
        cache_info.value = (
            f"<b>Cache:</b> {s['size']} entries | hits={s['hits']} | misses={s['misses']} | "
            f"eff={s['efficiency']:.1f}%"
        )

    def run_translate(_):
        """Execute translation and display results"""
        with output:
            clear_output()
            start = time.time()
            res = translator.translate(
                input_text.value,
                target_language=target_lang.value,
                source_language=source_lang.value,
                engine=engine.value,
            )
            dt = time.time() - start

            print(f"🌍 {res['src_emoji']} {res['source_language']} → {res['tgt_emoji']} {res['target_language']}")
            print("-" * 60)
            print("📤", res["original_text"])
            print()
            print("📥", res["translated_text"])
            print("-" * 60)
            print(f"⚙️ Engine: {res['engine_used']} | ⏱️ {dt:.2f}s | cached={res['cached']}")

        refresh_cache()

    def run_clear(_):
        """Clear input and output"""
        input_text.value = ""
        with output:
            clear_output()
        refresh_cache()

    def run_cache_clear(_):
        """Clear translation cache"""
        translator.cache.clear()
        refresh_cache()

    # Connect button events
    btn.on_click(run_translate)
    clear_btn.on_click(run_clear)
    cache_btn.on_click(run_cache_clear)

    refresh_cache()

    # Display the widget
    display(widgets.VBox([
        widgets.HTML("<h2>🌐 Universal Translator (Colab Ready)</h2>"),
        cache_info,
        input_text,
        widgets.HBox([source_lang, target_lang, engine]),
        widgets.HBox([btn, clear_btn, cache_btn]),
        output
    ]))

# Create and display the widget
create_widget()

## 5. Evaluation & Analysis

### a. Metrics Used
1. **Translation Quality**: Human evaluation through sample outputs
2. **Performance Metrics**:
   - Translation speed (seconds)
   - Cache efficiency (hit rate)
   - Engine selection accuracy
3. **System Metrics**:
   - GPU/CPU utilization
   - Memory efficiency with lazy loading

### b. Sample Outputs
The system provides detailed output including:
- Source and target languages with emojis
- Original and translated text
- Engine used for translation
- Translation time
- Cache status

### c. Performance Analysis
- **Speed**: Helsinki OPUS models are fastest but limited to specific pairs
- **Coverage**: NLLB-200 provides the widest language coverage
- **Accuracy**: Professional translation models ensure high quality
- **Scalability**: Caching system improves performance for repeated requests

### d. Limitations
1. Dependent on internet connection for model downloads
2. Limited to 512 characters per translation by default
3. Some language pairs may have lower quality than others
4. Real-time translation for very long texts requires optimization

## 6. Ethical Considerations & Responsible AI

### a. Bias and Fairness Considerations
1. **Language Bias**: The system supports 20+ languages but coverage varies
2. **Translation Quality**: Different engines may have varying accuracy for different languages
3. **Cultural Sensitivity**: Translations should consider cultural context

### b. Dataset Limitations
1. **Training Data**: Models trained on web-crawled data may contain biases
2. **Domain Specificity**: General models may not handle specialized terminology well
3. **Language Coverage**: Not all world languages are supported equally

### c. Responsible Use of AI Tools
1. **Transparency**: Clear indication of which engine is being used
2. **Fallback Mechanisms**: Multiple engines ensure reliability
3. **User Control**: Users can select specific engines or use auto-selection
4. **Error Handling**: Clear error messages when translation fails
5. **Privacy**: Local processing when possible, though models download from cloud

## 7. Conclusion & Future Scope

### a. Summary of Results
✅ **Successfully Implemented**:
- Multi-engine translation system with intelligent fallback
- Support for 20+ languages with auto-detection
- Interactive Colab-ready interface
- Performance optimization through caching
- GPU acceleration support

✅ **Key Features**:
- Three translation engines (Helsinki OPUS, NLLB-200, mBART-50)
- Automatic language detection
- Real-time translation with performance metrics
- Cache system for improved efficiency
- User-friendly widget interface

### b. Possible Improvements and Extensions
1. **Enhanced Features**:
   - Document translation (PDF, DOCX)
   - Batch file processing
   - Speech-to-speech translation
   
2. **Technical Enhancements**:
   - Larger context window for longer texts
   - Domain-specific fine-tuning
   - Local model deployment for offline use
   
3. **UI/UX Improvements**:
   - Web application interface
   - Mobile app version
   - API endpoint for integration
   
4. **Advanced Capabilities**:
   - Real-time translation streaming
   - Quality estimation scores
   - Alternative translation suggestions
   - Terminology customization

### c. Real-world Applications
1. **Education**: Language learning tool
2. **Business**: Multilingual communication
3. **Travel**: Real-time translation assistant
4. **Content Creation**: Multilingual content generation
5. **Research**: Cross-language information retrieval

---

**Project Status**: Fully Functional ✅  
**Ready for Deployment**: Yes  
**Scalability**: High (with GPU resources)  
**Accessibility**: Colab/Jupyter compatible  

This universal translator provides a robust, scalable solution for multilingual communication needs with professional-grade translation quality.